In [2]:
import socket, time

### Блокирующие сокеты

In [9]:
def get(url):
    sock = socket.socket()
    sock.connect(('google.com', 80))
    request = 'GET {} HTTP/1.1\r\nHost: google.com\r\n\r\n'.format(url)
    sock.send(bytes(request, encoding='utf-8'))
    response = b''
    chunk = sock.recv(4096)
    while chunk:
        response += chunk
        chunk = sock.recv(4096)
    
    return response.decode('utf-8').split()[:3]

In [10]:
start = time.time()

responses = []
urls = ['/', '/test/', '/404']
for url in urls:
    responses.append(get(url))
print(f"It took {time.time - start}")

TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'float'

### Неблокирующие сокеты

In [19]:
sock = socket.socket()
sock.setblocking(False)
sock.connect(('google.com', 80))

BlockingIOError: [Errno 115] Operation now in progress

In [ ]:
sock = socket.socket()
sock.setblocking(False)
try:
    sock.connect(('google.com', 80))
except BlockingIOError:
    pass

### Селекторы для неблокирующего ввода-вывода
Каждый селектор сугубо индивидуален для конкретной ОС. Для выбора наилучшего можно воспользоваться `DefaultSelector`

In [ ]:
from selectors import DefaultSelector, EVENT_WRITE
import socket

selector = DefaultSelector()

def get(url):
    sock = socket.socket()
    sock.setblocking(False)
    try:
        sock.connect(('google.com', 80))
    except BlockingIOError:
        pass
    selector.register(sock.fileno(), EVENT_WRITE)
    selector.select()
    selector.unregister(sock.fileno())
    request = 'GET {} HTTP/1.1\r\nHost: google.com\r\n\r\n'.format(url)
    sock.send(bytes(request, encoding='utf-8'))
    response = b''
    chunk = sock.recv(4096)
    while chunk:
        response += chunk
        chunk = sock.recv(4096)
    
    print(response)

In [7]:
from selectors import DefaultSelector, EVENT_WRITE, EVENT_READ
import socket

selector = DefaultSelector()

def get(url):
    sock = socket.socket()
    sock.setblocking(False)
    try:
        sock.connect(('google.com', 80))
    except BlockingIOError:
        pass
    selector.register(sock.fileno(), EVENT_WRITE)
    selector.select()
    request = 'GET {} HTTP/1.1\r\nHost: google.com\r\n\r\n'.format(url)
    selector.unregister(sock.fileno())
    sock.send(bytes(request, encoding='utf-8'))
    response = b''
    selector.register(sock.fileno(), EVENT_READ)
    selecor.select()
    selector.unregister(sock.fileno())
    chunk = sock.recv(4096)
    while chunk:
        response += chunk
        selector.register(sock.fileno(), EVENT_READ)
        selecor.select()
        selector.unregister(sock.fileno())
        chunk = sock.recv(4096)
    
    print(response)

## Callback!

In [ ]:
def on_readable(sock, request):
    selector.unregister(sock.fileno())
    sock.send(bytes(request, encoding='utf-8'))
    response = b''
    selector.register(sock.fileno(), EVENT_READ)
    selecor.select()
    selector.unregister(sock.fileno())
    chunk = sock.recv(4096)
    while chunk:
        response += chunk
        selector.register(sock.fileno(), EVENT_READ)
        selecor.select()
        selector.unregister(sock.fileno())
        chunk = sock.recv(4096)
    
    print(response)
    
def get(url):
    sock = socket.socket()
    sock.setblocking(False)
    try:
        sock.connect(('google.com', 80))
    except BlockingIOError:
        pass
    
    request = 'GET {} HTTP/1.1\r\nHost: google.com\r\n\r\n'.format(url)
    # Замыкание 
    callback = lambda: on_connected(sock, request)
    selector.register(sock.fileno(), EVENT_WRITE)
    selector.select()
    callback()



In [ ]:
while True:
    events = selector.select()
    
    for key, mask in events:
        callback = key.data
        callback()